### Explore the relationship between air quality and asthma at the county level for California

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

#### California air quality and asthma

In previous studies, I noted that sections of California have high PM2.5, ozone, and AQI indices. Unfortunately, there were no California asthma datasets by county for the years 1999, 2012, or 2020. California reports asthma in two-year periods, the most recent asthma data is for 2017 and 2018. For this study, I will read in 2017 and 2018 daily PM2.5 datasets, daily ozone datasets, and annual AQI datasets. I'll limit the datasets to California and follow procedures in previous studies for cleaning. Then I'll join the datasets to explore relationships between air quality and asthma.

### Read in datafiles, clean data, and print descriptive statistics

#### PM2.5

In [3]:
pm2017 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\daily_88101_2017.csv') 
pm2018 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\daily_88101_2018.csv') 

In [4]:
pm2017.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change'],
      dtype='object')

In [5]:
pm2017California = pm2017[pm2017['State Name'] == 'California']


In [6]:
pm2018California = pm2018[pm2018['State Name'] == 'California']

In [7]:
print(len(pm2017California))
print(len(pm2018California))

62774
66371


In [8]:
#pmCaliforniaCombined = pm2017California.append(pm2018California)Do SEPARATELY

Follow steps for cleaning previous PM2.5 files: <br>
(1) removal of all 1 hour samples, <br>
(2) removal of excluded event observations, <br>
(3) removal of 24-hour block averages when a site reports both 24-hour block average and 24 hour concentrations on the same date

In [23]:
pm2017California.groupby('Sample Duration').size()

Sample Duration
1 HOUR           26649
24 HOUR           9992
24-HR BLK AVG    26133
dtype: int64

In [24]:
pm2018California.groupby('Sample Duration').size()

Sample Duration
1 HOUR           28456
24 HOUR           9957
24-HR BLK AVG    27958
dtype: int64

In [25]:
#remove 1 hour samples
pm2017CaliforniaNon1HR = pm2017California[pm2017California['Sample Duration'] !=  '1 HOUR']
len(pm2017CaliforniaNon1HR)

36125

In [26]:
#remove 1 hour samples
pm2018CaliforniaNon1HR = pm2018California[pm2018California['Sample Duration'] !=  '1 HOUR']
len(pm2018CaliforniaNon1HR)

37915

In [27]:
#Check whether there are excluded observations in event type. 
pm2017CaliforniaNon1HR.groupby('Event Type').size()

Event Type
Excluded       64
Included     1884
None        34177
dtype: int64

In [28]:
#Check whether there are excluded observations in event type. 
pm2018CaliforniaNon1HR.groupby('Event Type').size()

Event Type
Excluded       20
Included     1818
None        36077
dtype: int64

In [30]:
#remove excluded event type observations
pm2017CaliforniaNon1HRNonExcluded = pm2017CaliforniaNon1HR[pm2017CaliforniaNon1HR['Event Type'] !=  'Excluded']
len(pm2017CaliforniaNon1HRNonExcluded)

36061

In [31]:
#remove excluded event type observations
pm2018CaliforniaNon1HRNonExcluded = pm2018CaliforniaNon1HR[pm2018CaliforniaNon1HR['Event Type'] !=  'Excluded']
len(pm2018CaliforniaNon1HRNonExcluded)

37895

In [32]:
#sort df
pm2017CaliforniaNon1HRNonExcludedSorted = pm2017CaliforniaNon1HRNonExcluded.sort_values(by=['County Code', 'Site Num', 'Date Local'])

In [33]:
#sort df
pm2018CaliforniaNon1HRNonExcludedSorted = pm2018CaliforniaNon1HRNonExcluded.sort_values(by=['County Code', 'Site Num', 'Date Local'])

In [35]:
#remove 24-hour block averages when a site reports both 24-hour block average and 24 hour concentrations on the same date
pmCalifornia2017Cleaned = pm2017CaliforniaNon1HRNonExcludedSorted.drop_duplicates(subset=['County Code','Site Num', 'Date Local'],
            keep='first')
len(pmCalifornia2017Cleaned)

30959

In [34]:
#remove 24-hour block averages when a site reports both 24-hour block average and 24 hour concentrations on the same date
pmCalifornia2018Cleaned = pm2018CaliforniaNon1HRNonExcludedSorted.drop_duplicates(subset=['County Code','Site Num', 'Date Local'],
            keep='first')
len(pmCalifornia2018Cleaned)

32573

In [37]:
#save cleaned dataset 
pmCalifornia2017Cleaned.to_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\daily_88101_2017_California_cleaned.csv', header = True)

In [38]:
#save cleaned dataset 
pmCalifornia2018Cleaned.to_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\daily_88101_2018_California_cleaned.csv', header = True)

In [39]:
#group by county
pmCalifornia2017County = pmCalifornia2017Cleaned['Arithmetic Mean'].groupby(pmCalifornia2017Cleaned['County Name']).mean()

In [40]:
#group by county
pmCalifornia2018County = pmCalifornia2018Cleaned['Arithmetic Mean'].groupby(pmCalifornia2018Cleaned['County Name']).mean()

In [41]:
pmCalifornia2017Countydf = pd.DataFrame(pmCalifornia2017County.to_frame().reset_index())
pmCalifornia2017Countydf

,County Name,Arithmetic Mean
0,Alameda,10.324092
1,Butte,9.036593
2,Calaveras,13.379149
3,Colusa,4.895823
4,Contra Costa,11.436072
5,Del Norte,5.993309
6,Fresno,11.894267
7,Humboldt,6.942453
8,Imperial,10.632168
9,Inyo,6.161150


In [42]:
pmCalifornia2018Countydf = pd.DataFrame(pmCalifornia2018County.to_frame().reset_index())
pmCalifornia2018Countydf

,County Name,Arithmetic Mean
0,Alameda,12.913035
1,Butte,13.773410
2,Calaveras,14.665546
3,Colusa,10.668367
4,Contra Costa,13.120255
5,Del Norte,7.825439
6,Fresno,13.732262
7,Humboldt,6.820779
8,Imperial,11.656298
9,Inyo,7.045304


In [43]:
#add year column
pmCalifornia2017Countydf['Year']= 2017

In [44]:
#add year column
pmCalifornia2018Countydf['Year']= 2018

In [46]:
#rename columns
pmCalifornia2017Countydf = pmCalifornia2017Countydf.rename(columns={'County Name': 'County', 'Arithmetic Mean': 'PM2.5 Mean'}).copy()

In [47]:
#rename columns
pmCalifornia2018Countydf = pmCalifornia2018Countydf.rename(columns={'County Name': 'County', 'Arithmetic Mean': 'PM2.5 Mean'}).copy()

In [48]:
pmCalifornia2017Countydf.describe()

,PM2.5 Mean,Year
count,46.000000,46.0
mean,9.660012,2017.0
std,2.844421,0.0
min,4.895823,2017.0
25%,7.837708,2017.0
50%,9.506551,2017.0
75%,11.517842,2017.0
max,16.725664,2017.0


In [49]:
pmCalifornia2018Countydf.describe()

,PM2.5 Mean,Year
count,47.000000,47.0
mean,11.352318,2018.0
std,3.178304,0.0
min,6.479295,2018.0
25%,9.189661,2018.0
50%,11.196875,2018.0
75%,13.016645,2018.0
max,20.871831,2018.0


In [ ]:
#TO DO: ADD commentary here

In [ ]:
#KEEP THIS SECTION FOR NOW> IT IS COMBINED BUT DOESN't HAVE YEARS

In [11]:
#Check whether there are multiple sample durations
#pm252011Kentucky.groupby('Sample Duration').size()
pmCaliforniaCombined.groupby('Sample Duration').size()

Sample Duration
1 HOUR           55105
24 HOUR          19949
24-HR BLK AVG    54091
dtype: int64

In [12]:
#remove 1 hour samples
pmCaliforniaCombinedNon1HR = pmCaliforniaCombined[pmCaliforniaCombined['Sample Duration'] !=  '1 HOUR']
len(pmCaliforniaCombinedNon1HR)

74040

In [13]:
#Check whether there are excluded observations in event type. 
pmCaliforniaCombinedNon1HR.groupby('Event Type').size()

Event Type
Excluded       84
Included     3702
None        70254
dtype: int64

In [14]:
#remove excluded event type observations
pmCaliforniaCombinedNon1HRNonExcluded = pmCaliforniaCombinedNon1HR[pmCaliforniaCombinedNon1HR['Event Type'] !=  'Excluded']
len(pmCaliforniaCombinedNon1HRNonExcluded)

73956

In [16]:
pmCaliforniaCombinedNon1HRNonExcludedSorted = pmCaliforniaCombinedNon1HRNonExcluded.sort_values(by=['County Code', 'Site Num', 'Date Local'])

In [17]:
#remove 24-hour block averages when a site reports both 24-hour block average and 24 hour concentrations on the same date
pmCaliforniaCleaned = pmCaliforniaCombinedNon1HRNonExcludedSorted.drop_duplicates(subset=['County Code','Site Num', 'Date Local'],
            keep='first')
len(pmCaliforniaCleaned)

63532

In [22]:
pmCaliforniaCleaned.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change'],
      dtype='object')

In [18]:
#save cleaned dataset 
pmCaliforniaCleaned.to_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\daily_88101_2017_2018_California_cleaned.csv', header = True)

In [19]:
#group by county
pmCaliforniaCounty = pmCaliforniaCleaned['Arithmetic Mean'].groupby(pmCaliforniaCleaned['County Name']).mean()

In [20]:
pmCaliforniaCountydf = pd.DataFrame(pmCaliforniaCounty.to_frame().reset_index())
pmCaliforniaCountydf

,County Name,Arithmetic Mean
0,Alameda,11.708453
1,Butte,11.508597
2,Calaveras,14.154899
3,Colusa,7.727910
4,Contra Costa,12.271067
5,Del Norte,6.538642
6,Fresno,12.819508
7,Humboldt,6.891257
8,Imperial,11.121807
9,Inyo,6.606611


In [50]:
pmCaliforniaCountydf.describe()

,Arithmetic Mean
count,47.000000
mean,10.580729
std,2.952941
min,6.054789
25%,8.283897
50%,10.671308
75%,12.088714
max,18.688235


#### Ozone

In [52]:
ozone2017 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\daily_44201_2017.csv') 
ozone2018 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\daily_44201_2018.csv') 

In [53]:
ozone2017California = ozone2017[ozone2017['State Name'] == 'California']

In [54]:
ozone2018California = ozone2018[ozone2018['State Name'] == 'California']

In [55]:
print(len(ozone2017California))
print(len(ozone2018California))

59262
60514


Follow steps for cleaning previous ozone files: <br>
(1) removal of excluded event observations, <br>

In [57]:
#Check whether there are excluded observations in event type
ozone2017California.groupby('Event Type').size()

Event Type
Excluded       13
Included     3452
None        55797
dtype: int64

In [58]:
#Check whether there are excluded observations in event type
ozone2018California.groupby('Event Type').size()

Event Type
Excluded        1
Included     3157
None        57356
dtype: int64

In [59]:
#remove excluded event type observations
ozone2017CaliforniaCleaned = ozone2017California[ozone2017California['Event Type'] !=  'Excluded']
len(ozone2017CaliforniaCleaned)

59249

In [60]:
#remove excluded event type observations
ozone2018CaliforniaCleaned = ozone2018California[ozone2018California['Event Type'] !=  'Excluded']
len(ozone2018CaliforniaCleaned)

60513

In [61]:
#save cleaned dataset 
ozone2017CaliforniaCleaned.to_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\daily_44201_2017_California_cleaned.csv', header = True)

In [62]:
#save cleaned dataset 
ozone2018CaliforniaCleaned.to_csv('C:\\Users\\Mary\\CIS5898\\FIT_capstone\\daily_44201_2018_California_cleaned.csv', header = True)

In [63]:
#group by county
ozone2017CaliforniaCounty = ozone2017CaliforniaCleaned['Arithmetic Mean'].groupby(ozone2017CaliforniaCleaned['County Name']).mean()

In [64]:
#group by county
ozone2018CaliforniaCounty = ozone2018CaliforniaCleaned['Arithmetic Mean'].groupby(ozone2018CaliforniaCleaned['County Name']).mean()

In [65]:
ozone2017CaliforniaCountydf = pd.DataFrame(ozone2017CaliforniaCounty.to_frame().reset_index())
ozone2017CaliforniaCountydf

,County Name,Arithmetic Mean
0,Alameda,0.023924
1,Amador,0.028316
2,Butte,0.037191
3,Calaveras,0.035239
4,Colusa,0.030274
5,Contra Costa,0.027160
6,El Dorado,0.043009
7,Fresno,0.035428
8,Glenn,0.029548
9,Humboldt,0.025919


In [66]:
ozone2018CaliforniaCountydf = pd.DataFrame(ozone2018CaliforniaCounty.to_frame().reset_index())
ozone2018CaliforniaCountydf

,County Name,Arithmetic Mean
0,Alameda,0.023922
1,Amador,0.029545
2,Butte,0.037431
3,Calaveras,0.033572
4,Colusa,0.028941
5,Contra Costa,0.026261
6,El Dorado,0.044532
7,Fresno,0.034762
8,Glenn,0.028012
9,Humboldt,0.024946


In [67]:
#add year column
ozone2017CaliforniaCountydf['Year']= 2017

In [68]:
#add year column
ozone2018CaliforniaCountydf['Year']= 2018

In [69]:
#rename columns
ozone2017CaliforniaCountydf = ozone2017CaliforniaCountydf.rename(columns={'County Name': 'County', 'Arithmetic Mean': 'Ozone Mean'}).copy()

In [70]:
#rename columns
ozone2018CaliforniaCountydf = ozone2018CaliforniaCountydf.rename(columns={'County Name': 'County', 'Arithmetic Mean': 'Ozone Mean'}).copy()

In [71]:
ozone2017CaliforniaCountydf.describe()

,Ozone Mean,Year
count,49.000000,49.0
mean,0.031650,2017.0
std,0.006395,0.0
min,0.018151,2017.0
25%,0.027566,2017.0
50%,0.030928,2017.0
75%,0.035239,2017.0
max,0.050560,2017.0


In [72]:
ozone2018CaliforniaCountydf.describe()

,Ozone Mean,Year
count,49.000000,49.0
mean,0.031420,2018.0
std,0.006061,0.0
min,0.021940,2018.0
25%,0.027031,2018.0
50%,0.030318,2018.0
75%,0.034762,2018.0
max,0.047137,2018.0


In [ ]:
#TO DO: ADD commentary here

#### AQI

In [73]:
AQI2017 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\annual_aqi_by_county_2017.csv') 
AQI2018 = pd.read_csv('C:\\Users\\Mary\\CIS5898\\annual_aqi_by_county_2018.csv') 

In [74]:
AQI2017.columns

Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10'],
      dtype='object')

In [75]:
AQI2017California = AQI2017[AQI2017['State'] == 'California']

In [76]:
AQI2018California = AQI2018[AQI2018['State'] == 'California']

In [77]:
print(len(AQI2017California))
print(len(AQI2018California))

53
53


In [79]:
AQI2017California.describe()

,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
count,53.0,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.0,53.000000,53.000000,53.000000,53.000000,53.000000
mean,2017.0,357.283019,205.264151,118.603774,24.867925,6.981132,1.037736,0.528302,288.301887,88.433962,49.037736,0.0,3.981132,212.849057,0.018868,122.962264,17.471698
std,0.0,30.233945,81.266454,55.456615,29.164017,11.551707,3.441613,2.232658,516.520183,31.554294,15.161898,0.0,9.030910,87.371971,0.137361,73.873323,49.092379
min,2017.0,203.000000,31.000000,10.000000,0.000000,0.000000,0.000000,0.000000,117.000000,44.000000,18.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.0,365.000000,152.000000,82.000000,6.000000,1.000000,0.000000,0.000000,154.000000,64.000000,39.000000,0.0,0.000000,172.000000,0.000000,94.000000,0.000000
50%,2017.0,365.000000,221.000000,111.000000,9.000000,3.000000,0.000000,0.000000,169.000000,82.000000,45.000000,0.0,0.000000,219.000000,0.000000,116.000000,2.000000
75%,2017.0,365.000000,273.000000,168.000000,33.000000,6.000000,0.000000,0.000000,197.000000,100.000000,54.000000,0.0,2.000000,253.000000,0.000000,155.000000,14.000000
max,2017.0,365.000000,350.000000,220.000000,115.000000,51.000000,23.000000,15.000000,3439.000000,185.000000,87.000000,0.0,47.000000,362.000000,1.000000,351.000000,341.000000


In [80]:
AQI2018California.describe()

,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
count,53.0,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000
mean,2018.0,359.207547,206.566038,118.377358,23.226415,9.679245,1.132075,0.226415,231.113208,87.320755,48.792453,0.037736,4.113208,201.641509,0.018868,138.679245,14.716981
std,0.0,27.214503,83.329630,60.999441,27.851006,9.281441,1.829646,0.669142,136.877635,27.802721,14.714408,0.192380,8.554487,88.386628,0.137361,75.463127,30.029074
min,2018.0,180.000000,35.000000,7.000000,0.000000,0.000000,0.000000,0.000000,77.000000,43.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.0,365.000000,153.000000,70.000000,5.000000,5.000000,0.000000,0.000000,166.000000,64.000000,39.000000,0.000000,0.000000,154.000000,0.000000,94.000000,0.000000
50%,2018.0,365.000000,219.000000,115.000000,14.000000,8.000000,0.000000,0.000000,196.000000,84.000000,45.000000,0.000000,0.000000,203.000000,0.000000,136.000000,1.000000
75%,2018.0,365.000000,275.000000,171.000000,26.000000,11.000000,2.000000,0.000000,240.000000,101.000000,54.000000,0.000000,2.000000,246.000000,0.000000,192.000000,12.000000
max,2018.0,365.000000,347.000000,257.000000,103.000000,51.000000,8.000000,4.000000,994.000000,164.000000,88.000000,1.000000,34.000000,364.000000,1.000000,350.000000,147.000000
